# Camera Calibration wit OpenCV
https://docs.opencv.org/4.5.3/dc/dbb/tutorial_py_calibration.html

In [4]:
import numpy as np
import cv2 as cv
import glob
# termination criteria

# Inner chessboard size (in number of squares)
cbSize = [5,8]

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((cbSize[0]*cbSize[1],3), np.float32)
objp[:,:2] = np.mgrid[0:cbSize[1],0:cbSize[0]].T.reshape(-1,2)*29
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('./data/iphone7/*.jpeg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (cbSize[1],cbSize[0]), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (cbSize[1],cbSize[0]), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

In [3]:
img

NameError: name 'img' is not defined

In [5]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [6]:
mtx

array([[1.32116357e+03, 0.00000000e+00, 5.99601040e+02],
       [0.00000000e+00, 1.31710545e+03, 8.05967335e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [18]:
img = cv.imread(images[0])
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [19]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

True

In [7]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.06732861750963728


In [8]:
dict(zip(['k1','k2','p1','p2','k3'],dist[0]))

{'k1': 0.26227771197588384,
 'k2': -1.7430035127963457,
 'p1': -0.00024315547759055133,
 'p2': 0.0005317115639280109,
 'k3': 3.3777201616621317}

{'k1': 0.2622777119756388,
 'k2': -1.7430035128242436,
 'p1': -0.0002431554773375689,
 'p2': 0.0005317115644495767,
 'k3': 3.3777201617889188}